# Reference:
* LeNet-5
  * [卷积神经网络Lenet-5详解](https://blog.csdn.net/d5224/article/details/68928083)
    * 這裡說明是用 Max Pooling, 並未提及 activation function 是用何者
  * [LeNet-5, convolutional neural networks](http://yann.lecun.com/exdb/lenet/)
    * LeNet-5 is our latest convolutional network designed for handwritten and machine-printed character recognition.
  * [LeNet-5 - A Classic CNN Architecture](https://engmrk.com/lenet-5-a-classic-cnn-architecture/)
    * 這裡說明是用 Avg Pooling, activation function 是 tanh 和 softmax
  * [PyTorch Taipei 2018 week1: LeNet-5](https://mattwang44.github.io/en/articles/PyTorchTP-LeNet/)
    * LeNet-5 介紹，詳盡的說明
  * [keras/examples/mnist_cnn.py](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)
    * 一個 CNN model，以 MNIST 為 dataset 的 example。以下實作的架構主要是參考此文  
  * [Netscope CNN Analyzer](https://dgschwend.github.io/netscope/quickstart.html)
    * 分析計算 CNN 的 operation 次數
  
* Dataset: MNIST
  * [Keras Documents: MNIST database of handwritten digits](https://keras.io/datasets/)
  * [THE MNIST DATABASE of handwritten digits](http://yann.lecun.com/exdb/mnist/)  
  
  
* Issues:
  * Wrokaround [Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR](https://github.com/tensorflow/tensorflow/issues/24496)

# Model Architecture
* Input Image: 28x28 (非原本 paper 的 32x32)
  * 將 pixel value 從 0~255 轉換成從 0.0~1.0
* Convolution (C1)
  * Filter: 6@5x5
  * Feature map: 6@28x28
  * 
* Pooling (S2)
* Convolution (C3)
* Pooling (S4)
* Convolution (C5)
* Fully-connected (F6)
* Output: Fully-connected (F7)

# Training and Evaluate

In [1]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("K.image_data_format() = ", K.image_data_format())
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Workaround Issue: "Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


K.image_data_format() =  channels_last
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)

# 分析計算量


| Layer | Ops |  Output|
|------|------|------|
| Image | None | 28x28 | 
| conv2d_1(32@3x3) | 32x(26x26)x(3x3x1)=194,688 macc | 32x(26x26) |
| relu | 32x(26x26)=21,632 comp | 32x(26x26) |
| conv2d_2(64@3x3) | 64x(24x24)x(3x3x1)=331,776 macc | 64x(24x24) |
| relu | 64x(24x24)=36,864 comp | 64x(24x24) |
| max_pooling2d_1(2x2) | 64x(12x12)x(2x2x1)=36,864 comp | 64x(12x12) |
| dropout_1(0.25) | None | 64x(12x12) |
| flatten_1 | None | 64x12x12 |
| dense_1(128) | 64x12x12x128=1197648 macc | 128 |
| dropout_2(0.25) | None | 128 |
| dense_2(10) | 128x10=1,280 macc | 10 |
| softmax | 10 softmax | 10 |